In [7]:
import gym
from gym import spaces
import numpy as np
import pandas as pd

class TradingEnv(gym.Env):
    def __init__(self):
        super(TradingEnv, self).__init__()

        # Load the CSV file with trading data
        self.data = pd.read_csv("C:\\Users\\Yash\\Downloads\\tata.csv")

        # Define the action space
        self.action_space = spaces.MultiDiscrete([3, 100])  # 3 actions: 0 (sell), 1 (hold), 2 (buy), with quantity 0-100

        # Define the observation space
        self.observation_space = spaces.Box(low=0, high=1, shape=(45, len(self.data.columns) - 1), dtype=np.float32)

        # Initialize other parameters
        self.balance = 100000
        self.shares_held = 0
        self.current_step = 45  # Start from the 45th candle
        self.reward_range = (-np.inf, np.inf)
        self.current_charge = 0
        self.profit = 0

    def reset(self):
        # Reset the environment to the initial state
        self.balance = 100000
        self.shares_held = 0
        self.current_step = 45  # Start from the 45th candle
        self.current_charge = 0
        self.profit = 0

        # Get the initial observation
        observation = self.get_observation()
        return observation

    def step(self, action):
        # Execute the selected action
        if action[0] == 0:  # Sell
            if self.shares_held >= action[1]:
                self.balance += action[1] * self.data.iloc[self.current_step]['Close']
                self.shares_held -= action[1]
                self.current_charge += 20
            else:
                return self.get_observation(), -10, True, {}

        elif action[0] == 2:  # Buy
            if self.balance >= action[1] * self.data.iloc[self.current_step]['Close']:
                self.balance -= action[1] * self.data.iloc[self.current_step]['Close']
                self.shares_held += action[1]
                self.current_charge += 100

        # Deduct charge every 90th candle
        if self.current_step % 90 == 0:
            self.balance -= 100

        # Check RSI condition
        if self.data.iloc[self.current_step]['RSI'] > 45 and action[0] == 2:  # Don't buy if RSI > 45
            return self.get_observation(), -5, True, {}

        # Update the current step
        self.current_step += 1

        # Calculate the reward
        reward = self.calculate_reward()

        # Check if the episode is done
        done = self.current_step == len(self.data) - 1 or self.balance <= 0

        return self.get_observation(), reward, done, {}

    def get_observation(self):
        # Get the current observation (past 45 candle data)
        observation = self.data.iloc[self.current_step - 45:self.current_step, :-1].values
        return observation

    def calculate_reward(self):
        # Calculate the reward based on profit/loss
        current_value = self.balance + self.shares_held * self.data.iloc[self.current_step]['Close']
        reward = current_value - 100000 - self.current_charge
        return reward

    def render(self):
        # Display current balance and shares held
        print(f"Balance: {self.balance}, Shares Held: {self.shares_held}")

# Test the environment
if __name__ == "__main__":
    env = TradingEnv()
    observation = env.reset()
    done = False
    while not done:
        action = env.action_space.sample()  # Random action for testing
        observation, reward, done, _ = env.step(action)
        env.render()


Balance: 100000, Shares Held: 0
Balance: 99340.91382029, Shares Held: 70


In [8]:
MultiDiscrete([3, 100])

NameError: name 'MultiDiscrete' is not defined

In [11]:
import gym
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv


# Import your custom TradingEnv (update the import statement if needed)
# from trading_env import TradingEnv  # Make sure to replace 'trading_env' with your file name

# Create a function to evaluate the trained model
def evaluate_trained_model(model, env, num_episodes=10):
    mean_rewards, std_rewards = evaluate_policy(model, env, n_eval_episodes=num_episodes)
    return mean_rewards, std_rewards

# Create and wrap the custom environment
env = TradingEnv()
env = DummyVecEnv([lambda: env])

# Create the PPO agent
model = PPO("MlpPolicy", env, verbose=1)

# Train the model (you can adjust the number of training steps)
model.learn(total_timesteps=1000000)

# Evaluate the trained model
mean_rewards, std_rewards = evaluate_trained_model(model, env)

print(f"Mean Reward: {mean_rewards:.2f}, Std Reward: {std_rewards:.2f}")

# Test the model for a few episodes
for _ in range(5):
    obs = env.reset()
    done = False
    total_reward = 0
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, _ = env.step(action)
        total_reward += reward
        env.render()
    print(f"Episode Reward: {total_reward:.2f}")


C:\Users\Yash\AppData\Roaming\Python\Python310\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cpu device
-----------------------------
| time/              |      |
|    fps             | 833  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 654        |
|    iterations           | 2          |
|    time_elapsed         | 6          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.19097495 |
|    clip_fraction        | 0.69       |
|    clip_range           | 0.2        |
|    entropy_loss         | -5.6       |
|    explained_variance   | -0.0082    |
|    learning_rate        | 0.0003     |
|    loss                 | 3.05       |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.121     |
|    value_loss           | 30.7       |
----------------------------------------
-----------------------------------

-------------------------------------------
| time/                   |               |
|    fps                  | 578           |
|    iterations           | 13            |
|    time_elapsed         | 46            |
|    total_timesteps      | 26624         |
| train/                  |               |
|    approx_kl            | 0.00014299608 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -4.77         |
|    explained_variance   | -0.000384     |
|    learning_rate        | 0.0003        |
|    loss                 | 3.35e+04      |
|    n_updates            | 120           |
|    policy_gradient_loss | -0.000482     |
|    value_loss           | 4.73e+04      |
-------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 578          |
|    iterations           | 14           |
|    time_elapsed         | 49      

------------------------------------------
| time/                   |              |
|    fps                  | 580          |
|    iterations           | 24           |
|    time_elapsed         | 84           |
|    total_timesteps      | 49152        |
| train/                  |              |
|    approx_kl            | 2.876157e-06 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -4.63        |
|    explained_variance   | 0.000672     |
|    learning_rate        | 0.0003       |
|    loss                 | 1.79e+05     |
|    n_updates            | 230          |
|    policy_gradient_loss | -0.000259    |
|    value_loss           | 6.38e+05     |
------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 580           |
|    iterations           | 25            |
|    time_elapsed         | 88            |
|    t

-----------------------------------------
| time/                   |             |
|    fps                  | 580         |
|    iterations           | 35          |
|    time_elapsed         | 123         |
|    total_timesteps      | 71680       |
| train/                  |             |
|    approx_kl            | 0.020256128 |
|    clip_fraction        | 0.242       |
|    clip_range           | 0.2         |
|    entropy_loss         | -4.56       |
|    explained_variance   | -0.388      |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0713     |
|    n_updates            | 340         |
|    policy_gradient_loss | -0.0433     |
|    value_loss           | 0.31        |
-----------------------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 580        |
|    iterations           | 36         |
|    time_elapsed         | 126        |
|    total_timesteps      | 73728      

...

-----------------------------------------
| time/                   |             |
|    fps                  | 572         |
|    iterations           | 464         |
|    time_elapsed         | 1659        |
|    total_timesteps      | 950272      |
| train/                  |             |
|    approx_kl            | 0.025692526 |
|    clip_fraction        | 0.251       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.73       |
|    explained_variance   | -0.11       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0804      |
|    n_updates            | 4630        |
|    policy_gradient_loss | -0.0213     |
|    value_loss           | 0.263       |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 572         |
|    iterations           | 465         |
|    time_elapsed         | 1663        |
|    total_timesteps      | 952320

-------------------------------------------
| time/                   |               |
|    fps                  | 572           |
|    iterations           | 475           |
|    time_elapsed         | 1699          |
|    total_timesteps      | 972800        |
| train/                  |               |
|    approx_kl            | 1.4260877e-09 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -2.04         |
|    explained_variance   | 0.13          |
|    learning_rate        | 0.0003        |
|    loss                 | 2.17e+07      |
|    n_updates            | 4740          |
|    policy_gradient_loss | -1.1e-05      |
|    value_loss           | 4.51e+07      |
-------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 572          |
|    iterations           | 476          |
|    time_elapsed         | 1703    

----------------------------------------
| time/                   |            |
|    fps                  | 572        |
|    iterations           | 486        |
|    time_elapsed         | 1739       |
|    total_timesteps      | 995328     |
| train/                  |            |
|    approx_kl            | 0.07381343 |
|    clip_fraction        | 0.298      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.84      |
|    explained_variance   | -162       |
|    learning_rate        | 0.0003     |
|    loss                 | 0.0362     |
|    n_updates            | 4850       |
|    policy_gradient_loss | -0.0284    |
|    value_loss           | 2.98       |
----------------------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 572        |
|    iterations           | 487        |
|    time_elapsed         | 1742       |
|    total_timesteps      | 997376     |
| train/        

C:\Users\Yash\AppData\Roaming\Python\Python310\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
C:\Users\Yash\AppData\Roaming\Python\Python310\site-packages\stable_baselines3\common\vec_env\base_vec_env.py:234: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


TypeError: unsupported format string passed to numpy.ndarray.__format__

In [5]:
pip install stable-baselines3[extra]


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
